In [1]:
import sys, os, math
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx import print_matrix
from pyzx.basicrules import *
from fractions import Fraction
import random

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [2]:
def unmap_qubit(p, width):
    y,z = p
    return y + z*width

def map_qubits(g, width):
    for v in g.vertices():
        q = g.qubit(v)
        z = math.floor(q/width)
        y = q - z*width
        g.set_qubit(v, y)
        g.set_vdata(v, 'z', z)

def random_nn_cnots(qubits, width, n):
    height = math.floor(qubits/width)
    c = zx.Circuit(qubits)
    g = 0
    for _ in range(n):
        q = (random.randint(0,height-1), random.randint(0,width-1))
        ns = [(q[0]-1, q[1]), (q[0]+1, q[1]), (q[0], q[1]-1), (q[0], q[1]+1)]
        q1 = random.choice([(y,z) for y,z in ns if y >= 0 and y < height and z >= 0 and z < width])
        c.add_gate('CNOT', unmap_qubit(q, width), unmap_qubit(q1, width))
    return c

In [3]:
qubits = 9
width = 3
gates = 20

random.seed(1337)
c = random_nn_cnots(qubits, width, gates)
print(c.gates)
g = c.to_graph()
map_qubits(g, width)

[CNOT(8,5), CNOT(8,7), CNOT(4,1), CNOT(4,5), CNOT(5,4), CNOT(7,8), CNOT(6,3), CNOT(4,1), CNOT(2,5), CNOT(2,5), CNOT(0,3), CNOT(8,5), CNOT(5,4), CNOT(2,1), CNOT(3,6), CNOT(2,1), CNOT(2,1), CNOT(3,4), CNOT(7,4), CNOT(1,4)]


In [4]:
zx.draw_3d(g)

In [5]:
def connect(g, vs, ws):
    if len(vs) == 0: return
    for i in range(len(vs)):
        for j in range(len(vs[0])):
            g.add_edge((vs[i][j], ws[i][j]))

def add_boundary(g, x, yrange, zrange):
    vs = [[g.add_vertex(B, row=x, qubit=y) for _ in zrange] for y in yrange]
    for y in yrange:
        for z in zrange:
            g.set_vdata(vs[y][z], 'z', z)
    return vs

def add_patch(g, ty, x, yrange, zrange, orientation='H'):
    ty_op = X if ty == Z else Z
    vs = [[g.add_vertex(ty, row=x, qubit=y) for z in zrange] for y in yrange]
    parity = 0 if ty == Z else 1
    for y in range(yrange.start, yrange.stop+1):
        for z in range(zrange.start, zrange.stop+1):
            if y in yrange and z in zrange: g.set_vdata(vs[y][z], 'z', z)
            if orientation == 'H' and (y == yrange.start or y == yrange.stop): continue
            if orientation == 'V' and (z == zrange.start or z == zrange.stop): continue
            if (y+z) % 2 == parity:
                w = g.add_vertex(ty_op, row=x, qubit=y-0.5)
                g.set_vdata(w, 'z', z-0.5)
                for (i,j) in ((y-1, z-1), (y, z-1), (y-1, z), (y, z)):
                    if i in yrange and j in zrange:
                        g.add_edge((w, vs[i][j]))
    return vs

In [9]:
g = zx.Graph()
sz = range(0,5)
x = 0
p1 = add_boundary(g, x, sz, sz); x += 2
for _ in range(2):
    p = add_patch(g, Z, x, sz, sz, 'H'); x += 1
    connect(g, p1, p)
    p1 = add_patch(g, X, x, sz, sz, 'V'); x += 2
    connect(g, p, p1)
p = add_boundary(g, x, sz, sz)
connect(g, p1, p)

zx.draw_3d(g)